<h1 align=center><font size = 5>TOURISM IN SARDINIA</font></h1>

## Introduction

The aim of this work is to analyze tourism in North Sardinia and try to suggest possible actions to react to Covid-19 crisis. The problem to solve is why some areas, like Costa Smeralda in the North East of the island, can attract a lot of tourists and some others are almost unknown even with beautiful landscape and environmental attractions? 
Maybe this is due to how the touristic offer is composed and so I analyzed venues and accomodation facilities in each municipality in the North part of Sardinia: the Province of Sassari.

First let's start importing the libraries we will need.

In [0]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install wikipedia
!pip install BeautifulSoup4
!pip install requests
!pip install lxml

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

from bs4 import BeautifulSoup
from pandas import DataFrame
import wikipedia
import wikipedia as wp

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.')

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=8d4c80343ecbc8ed2d798d0c971347f39429ca2bb710fcc14b7c9d097ad9e34b
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
/bin/bash: conda: command not found
/bin/bash: conda: command not found
Libraries imported.


## Initiating

Then we import from a Wikipedia page the list of the municipalities.
We have had to replace the name of Sassari as Sassari,SS to make no confusing the city with the province.

In [0]:
url = requests.get('https://it.wikipedia.org/wiki/Comuni_della_Sardegna').text
soup = BeautifulSoup(url,'lxml')
table_post = soup.find('table')
fields = table_post.find_all('td')
comune = []
popolazione = []
provincia = []

for i in range(0, len(fields),5):
 comune.append(fields[i].text.strip())
 provincia.append(fields[2+i].text.strip())
 popolazione.append(fields[3+i].text.strip())

pop = pd.DataFrame(data=[comune, provincia, popolazione]).transpose()
pop.columns = ['City', 'Province', 'Population']
pop['City'] = pop['City'].str.upper() 
pop['Province'] = pop['Province'].str.upper() 
pop = pop[pop.Province == 'SASSARI'].reset_index(drop=True)
pop.City.replace(['SASSARI'],'SASSARI, SS', inplace=True)
pop.tail(3)

,City,Province,Population
89,VALLEDORIA,SASSARI,4358
90,VIDDALBA,SASSARI,1668
91,VILLANOVA MONTELEONE,SASSARI,2227


With this list we used geopy library to sort out latidute and longitude of each municipality.

In [0]:
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim 

# declare an empty list to store 
# latitude and longitude of values 
# of city column 
longitude = [] 
latitude = [] 

# function to find the coordinate 
# of a given city 
def findGeocode(City): 
	
	# try and catch is used to overcome 
	# the exception thrown by geolocator 
	# using geocodertimedout 
	try: 
		
		# Specify the user_agent as your 
		# app name it should not be none 
		geolocator = Nominatim(user_agent="your_app_name") 
		
		return geolocator.geocode(City) 
	
	except GeocoderTimedOut: 
		
		return findGeocode(City)	 

# each value from city column 
# will be fetched and sent to 
# function find_geocode 
for i in (pop["City"]+","+"Sardinia"): 
	
	if findGeocode(i) != None: 
		
		loc = findGeocode(i) 
		
		# coordinates returned from 
		# function is stored into 
		# two seperate list 
		latitude.append(loc.latitude) 
		longitude.append(loc.longitude) 
	
	# if coordinate for a city not 
	# found, insert "NaN" indicating 
	# missing value 
	else: 
		latitude.append(np.nan) 
		longitude.append(np.nan) 

In [0]:
# now add this column to dataframe 
pop["Longitude"] = longitude 
pop["Latitude"] = latitude 
pop.head(3)


,City,Province,Population,Longitude,Latitude
0,AGGIUS,SASSARI,1436,9.065222,40.929345
1,AGLIENTU,SASSARI,1221,9.112719,41.079573
2,ALÀ DEI SARDI,SASSARI,1846,9.327829,40.650066


Using the matplotlib library we pointed out the cities on the map.

In [0]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# create map using latitude and longitude values
map_city = folium.Map(location=[latitude, longitude], zoom_start=8)
   
# add markers to map
for latitude, longitude, City in zip(pop['Latitude'], pop['Longitude'], pop['City']):
    label = '{}'.format(City)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_city)

map_city

So we used the Foursquare API to investigate venues around each municipality in a round of 1.5 Km.

In [0]:
CLIENT_ID = 'Y5ASVFKYM2KGEMFOJOUHDUS5VVJUSOBARQFIJCOEW0TCH4SQ' # your Foursquare ID
CLIENT_SECRET = 'GMRKBCRHFKYQJQELIEIXP33SPVXJREMOL4SIY0D1YEDIXR3B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y5ASVFKYM2KGEMFOJOUHDUS5VVJUSOBARQFIJCOEW0TCH4SQ
CLIENT_SECRET:GMRKBCRHFKYQJQELIEIXP33SPVXJREMOL4SIY0D1YEDIXR3B


We explored the first city in our dataframe to test the process.

In [0]:
pop.loc[0, 'City']

'AGGIUS'

We found the city's latitude and longitude values.

In [0]:
city_latitude = pop.loc[0, 'Latitude'] # city latitude value
city_longitude = pop.loc[0, 'Longitude'] # city longitude value

city_name = pop.loc[0, 'City'] # city name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))

Latitude and longitude values of AGGIUS are 40.9293453, 9.0652223.


Then we did the same with the top 100 venues that are in Aggius within a radius of 1500 meters.

We created the GET request URL. Name your URL **url**.

In [0]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=Y5ASVFKYM2KGEMFOJOUHDUS5VVJUSOBARQFIJCOEW0TCH4SQ&client_secret=GMRKBCRHFKYQJQELIEIXP33SPVXJREMOL4SIY0D1YEDIXR3B&v=20180605&ll=40.9293453,9.0652223&radius=1500&limit=100'

and then sent the GET request and examine the results.

In [0]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed8c27d0cc1fd001b0f49f7'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4e06357c1838eb15aef97728-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d110941735',
         'name': 'Italian Restaurant',
         'pluralName': 'Italian Restaurants',
         'primary': True,
         'shortName': 'Italian'}],
       'id': '4e06357c1838eb15aef97728',
       'location': {'cc': 'IT',
        'country': 'Italia',
        'distance': 860,
        'formattedAddress': ['Sardegna', 'Italia'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.92201300674671,
          'lng': 9.068466501906517}],
        'lat': 40.92201300674671,
        'lng': 9.068466501

We borrowed the **get_category_type** function from the Foursquare lab.

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

At this point, we were ready to clean the json and structure it into a *pandas* dataframe.

In [0]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Il Muto Di Gallura,Italian Restaurant,40.922013,9.068467
1,Museo Aggius,History Museum,40.928497,9.065421
2,Bar GeGe,Bar,40.930775,9.064972
3,Trattoria La Pavoncella,Italian Restaurant,40.930987,9.064776
4,Museo del Banditismo,History Museum,40.928720,9.065686


And how many venues were returned by Foursquare?

In [0]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


## Explore cities in the Province of Sassari

#### Let's create a function to process all the cities in Province of Sassari

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Lat', 
                  'Venue Long', 
                  'Venue Category']
    
    return(nearby_venues)

Now the code to run the above function on each neighborhood and create a new dataframe called venues.

In [0]:
province_venues = getNearbyVenues(names=pop['City'],
                                   latitudes=pop['Latitude'],
                                   longitudes=pop['Longitude']
                                  )

AGGIUS
AGLIENTU
ALÀ DEI SARDI
ALGHERO
ANELA
ARDARA
ARZACHENA
BADESI
BANARI
BENETUTTI
BERCHIDDA
BESSUDE
BONNANARO
BONO
BONORVA
BORTIGIADAS
BORUTTA
BOTTIDDA
BUDDUSÒ
BUDONI
BULTEI
BULZI
BURGOS
CALANGIANUS
CARGEGHE
CASTELSARDO
CHEREMULE
CHIARAMONTI
CODRONGIANOS
COSSOINE
ERULA
ESPORLATU
FLORINAS
GIAVE
GOLFO ARANCI
ILLORAI
ITTIREDDU
ITTIRI
LA MADDALENA
LAERRU
LOIRI PORTO SAN PAOLO
LUOGOSANTO
LURAS
MARA
MARTIS
MONTELEONE ROCCA DORIA
MONTI
MORES
MUROS
NUGHEDU SAN NICOLÒ
NULE
NULVI
OLBIA
OLMEDO
OSCHIRI
OSILO
OSSI
OZIERI
PADRIA
PADRU
PALAU
PATTADA
PERFUGAS
PLOAGHE
PORTO TORRES
POZZOMAGGIORE
PUTIFIGARI
ROMANA
SAN TEODORO
SANTA MARIA COGHINAS
SANTA TERESA GALLURA
SANT'ANTONIO DI GALLURA
SASSARI, SS
SEDINI
SEMESTENE
SENNORI
SILIGO
SORSO
STINTINO
TELTI
TEMPIO PAUSANIA
TERGU
THIESI
TISSI
TORRALBA
TRINITÀ D'AGULTU E VIGNOLA
TULA
URI
USINI
VALLEDORIA
VIDDALBA
VILLANOVA MONTELEONE


#### Let's check the size of the resulting dataframe

In [0]:
print(province_venues.shape)
province_venues

(578, 7)


,City,Latitude,Longitude,Venue,Venue Lat,Venue Long,Venue Category
0,AGGIUS,40.929345,9.065222,Il Muto Di Gallura,40.922013,9.068467,Italian Restaurant
1,AGGIUS,40.929345,9.065222,Museo Aggius,40.928497,9.065421,History Museum
2,AGGIUS,40.929345,9.065222,Bar GeGe,40.930775,9.064972,Bar
3,AGGIUS,40.929345,9.065222,Trattoria La Pavoncella,40.930987,9.064776,Italian Restaurant
4,AGGIUS,40.929345,9.065222,Museo del Banditismo,40.928720,9.065686,History Museum
5,AGGIUS,40.929345,9.065222,Laggetto,40.928362,9.058199,Lake
6,AGGIUS,40.929345,9.065222,Antiga Pizzaria in Aggius,40.934972,9.066752,Pizza Place
7,AGLIENTU,41.079573,9.112719,B&B Brezza Marina,41.079214,9.113095,Bed & Breakfast
8,AGLIENTU,41.079573,9.112719,Da Piera,41.080528,9.114146,Gourmet Shop
9,ALÀ DEI SARDI,40.650066,9.327829,Impresa Edile Edil Bua Sas,40.648410,9.328071,Construction & Landscaping


Let's check how many venues were returned for each municipality.

In [0]:
province_venues.groupby('City').count()

,Latitude,Longitude,Venue,Venue Lat,Venue Long,Venue Category
City,,,,,,
AGGIUS,7,7,7,7,7,7
AGLIENTU,2,2,2,2,2,2
ALGHERO,89,89,89,89,89,89
ALÀ DEI SARDI,1,1,1,1,1,1
ANELA,1,1,1,1,1,1
ARDARA,2,2,2,2,2,2
ARZACHENA,5,5,5,5,5,5
BADESI,5,5,5,5,5,5
BANARI,2,2,2,2,2,2


Let's find out how many unique categories can be curated from all the returned venues

In [0]:
print('There are {} uniques categories.'.format(len(province_venues['Venue Category'].unique())))

There are 111 uniques categories.


## Analyzing each city

In [0]:
# one hot encoding
pop_onehot = pd.get_dummies(province_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
pop_onehot['City'] = province_venues['City'] 

# move city column to the first column
fixed_columns = [pop_onehot.columns[-1]] + list(pop_onehot.columns[:-1])
pop_onehot = pop_onehot[fixed_columns]

pop_onehot.head()

,Wine Shop,Agriturismo,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Garage,Bakery,Bar,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Boat or Ferry,Bookstore,Brewery,Business Service,Butcher,Cafeteria,Café,Castle,Cheese Shop,City,Clothing Store,Cocktail Bar,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Food Court,Funeral Home,Furniture / Home Store,Garden,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Harbor / Marina,Hardware Store,Historic Site,History Museum,Home Service,Hot Spring,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Locksmith,Lounge,Market,Mediterranean Restaurant,Mini Golf,Mountain,Movie Theater,National Park,Nightclub,Non-Profit,Outdoor Sculpture,Paella Restaurant,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Plaza,Port,Pub,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Sandwich Place,Sardinian Restaurant,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Supermarket,Surf Spot,Tapas Restaurant,Theater,Train Station,Transportation Service,Trattoria/Osteria
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AGGIUS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AGGIUS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AGGIUS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AGGIUS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AGGIUS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [0]:
pop_onehot.shape

(578, 111)

#### Next, let's group rows by city and by taking the mean of the frequency of occurrence of each category

In [0]:
pop_grouped = pop_onehot.groupby('City').mean().reset_index()
pop_grouped

,City,Wine Shop,Agriturismo,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Garage,Bakery,Bar,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Boat or Ferry,Bookstore,Brewery,Business Service,Butcher,Cafeteria,Café,Castle,Cheese Shop,Clothing Store,Cocktail Bar,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Food Court,Funeral Home,Furniture / Home Store,Garden,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Harbor / Marina,Hardware Store,Historic Site,History Museum,Home Service,Hot Spring,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Locksmith,Lounge,Market,Mediterranean Restaurant,Mini Golf,Mountain,Movie Theater,National Park,Nightclub,Non-Profit,Outdoor Sculpture,Paella Restaurant,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Plaza,Port,Pub,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Sandwich Place,Sardinian Restaurant,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Supermarket,Surf Spot,Tapas Restaurant,Theater,Train Station,Transportation Service,Trattoria/Osteria
0,AGGIUS,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.285714,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.285714,0.000000,0.00,0.000000,0.000000,0.142857,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000
1,AGLIENTU,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000
2,ALGHERO,0.011236,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.011236,0.022472,0.033708,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.011236,0.00,0.000000,0.00,0.078652,0.00,0.00,0.000000,0.011236,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.011236,0.00,0.000000,0.0,0.011236,0.000000,0.011236,0.000000,0.000000,0.000000,0.0,0.000000,0.011236,0.011236,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.101124,0.000000,0.000000,0.00,0.033708,0.179775,0.011236,0.00,0.000000,0.000000,0.000000,0.000000,0.022472,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.011236,0.00,0.000000,0.000000,0.089888,0.022472,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.078652,0.0,0.022472,0.011236,0.011236,0.056180,0.000000,0.000000,0.00,0.000000,0.01123

#### Let's confirm the new size

In [0]:
pop_grouped.shape

(82, 111)

#### Let's print each city along with the top 10 most common venues

In [0]:
num_top_venues = 10

for hood in pop_grouped['City']:
    print("----"+hood+"----")
    temp = pop_grouped[pop_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AGGIUS----
                           venue  freq
0             Italian Restaurant  0.29
1                 History Museum  0.29
2                    Pizza Place  0.14
3                            Bar  0.14
4                           Lake  0.14
5                      Wine Shop  0.00
6                       Mountain  0.00
7                       Pharmacy  0.00
8                           Park  0.00
9  Paper / Office Supplies Store  0.00


----AGLIENTU----
                           venue  freq
0                Bed & Breakfast   0.5
1                   Gourmet Shop   0.5
2                      Wine Shop   0.0
3                      Mini Golf   0.0
4                    Pizza Place   0.0
5                       Pharmacy   0.0
6                           Park   0.0
7  Paper / Office Supplies Store   0.0
8              Paella Restaurant   0.0
9              Outdoor Sculpture   0.0


----ALGHERO----
                venue  freq
0  Italian Restaurant  0.18
1               Hotel  0.10
2     

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each city.

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['City'] = pop_grouped['City']

for ind in np.arange(pop_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(pop_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AGGIUS,History Museum,Italian Restaurant,Bar,Lake,Pizza Place,Food,Dessert Shop,Diner,Discount Store,Dive Bar
1,AGLIENTU,Gourmet Shop,Bed & Breakfast,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm
2,ALGHERO,Italian Restaurant,Hotel,Pizza Place,Restaurant,Café,Trattoria/Osteria,Seafood Restaurant,Beach Bar,Ice Cream Shop,Supermarket
3,ALÀ DEI SARDI,Construction & Landscaping,Trattoria/Osteria,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant
4,ANELA,Bar,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant


## Clustering the cities

Run *k*-means to cluster the municipalities into 5 clusters.

In [0]:
# set number of clusters
kclusters = 5

pop_grouped_clustering = pop_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pop_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 2, 4, 1, 1, 1, 1, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each city.

In [0]:
# add clustering labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

pop_merged = pop

# merge pop_grouped with pop to add latitude/longitude for each city
pop_merged = pop_merged.join(venues_sorted.set_index('City'), on='City')
pop_merged.dropna(subset=['Cluster Labels'],inplace=True)
pop_merged[["Cluster Labels"]] = pop_merged[["Cluster Labels"]].astype("int")
pop_merged.head(90) # check the last columns!

,City,Province,Population,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AGGIUS,SASSARI,1436,9.065222,40.929345,1,History Museum,Italian Restaurant,Bar,Lake,Pizza Place,Food,Dessert Shop,Diner,Discount Store,Dive Bar
1,AGLIENTU,SASSARI,1221,9.112719,41.079573,1,Gourmet Shop,Bed & Breakfast,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm
2,ALÀ DEI SARDI,SASSARI,1846,9.327829,40.650066,2,Construction & Landscaping,Trattoria/Osteria,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant
3,ALGHERO,SASSARI,43845,8.315147,40.560021,1,Italian Restaurant,Hotel,Pizza Place,Restaurant,Café,Trattoria/Osteria,Seafood Restaurant,Beach Bar,Ice Cream Shop,Supermarket
4,ANELA,SASSARI,615,9.057427,40.441760,4,Bar,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant
5,ARDARA,SASSARI,767,8.810319,40.621761,1,Café,Restaurant,Trattoria/Osteria,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm
6,ARZACHENA,SASSARI,13816,9.387612,41.079898,1,Italian Restaurant,Hotel,Café,Shopping Mall,Hardware Store,Flower Shop,Dessert Shop,Diner,Discount Store,Dive Bar
7,BADESI,SASSARI,1 862,8.884216,40.965069,1,Restaurant,Transportation Service,Ice Cream Shop,Pizza Place,Trattoria/Osteria,Flower Shop,Department Store,Dessert Shop,Diner,Discount Store
8,BANARI,SASSARI,550,8.697371,40.572927,1,Rental Car Location,Bed & Breakfast,Trattoria/Osteria,Food,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm
9,BENETUTTI,SASSARI,1793,9.168829,40.456461,2,Construction & Landscaping,Pizza Place,Flower Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store


Finally, let's visualize the resulting clusters.

In [0]:
pop_merged[['Cluster Labels']] = pop_merged[['Cluster Labels']].astype("int")
pop_merged.dtypes

City                       object
Province                   object
Population                 object
Longitude                 float64
Latitude                  float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [0]:
pop_merged.head(90)

,City,Province,Population,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AGGIUS,SASSARI,1436,9.065222,40.929345,1,History Museum,Italian Restaurant,Bar,Lake,Pizza Place,Food,Dessert Shop,Diner,Discount Store,Dive Bar
1,AGLIENTU,SASSARI,1221,9.112719,41.079573,1,Gourmet Shop,Bed & Breakfast,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm
2,ALÀ DEI SARDI,SASSARI,1846,9.327829,40.650066,2,Construction & Landscaping,Trattoria/Osteria,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant
3,ALGHERO,SASSARI,43845,8.315147,40.560021,1,Italian Restaurant,Hotel,Pizza Place,Restaurant,Café,Trattoria/Osteria,Seafood Restaurant,Beach Bar,Ice Cream Shop,Supermarket
4,ANELA,SASSARI,615,9.057427,40.441760,4,Bar,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant
5,ARDARA,SASSARI,767,8.810319,40.621761,1,Café,Restaurant,Trattoria/Osteria,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm
6,ARZACHENA,SASSARI,13816,9.387612,41.079898,1,Italian Restaurant,Hotel,Café,Shopping Mall,Hardware Store,Flower Shop,Dessert Shop,Diner,Discount Store,Dive Bar
7,BADESI,SASSARI,1 862,8.884216,40.965069,1,Restaurant,Transportation Service,Ice Cream Shop,Pizza Place,Trattoria/Osteria,Flower Shop,Department Store,Dessert Shop,Diner,Discount Store
8,BANARI,SASSARI,550,8.697371,40.572927,1,Rental Car Location,Bed & Breakfast,Trattoria/Osteria,Food,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm
9,BENETUTTI,SASSARI,1793,9.168829,40.456461,2,Construction & Landscaping,Pizza Place,Flower Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store


and finally we showed results on a map

In [0]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pop_merged['Latitude'], pop_merged['Longitude'], pop_merged['City'], pop_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 
In general, we can say that there the great part of the cities in North Sardinia have the same offering structures of the most attractive ones.

Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

Very small town where venues are mainly dedicated to daily food provisioning.

In [0]:
pop_merged.loc[pop_merged['Cluster Labels'] == 0, pop_merged.columns[[0] + list(range(5, pop_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,BOTTIDDA,0,Park,Trattoria/Osteria,Food,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant
22,BURGOS,0,Park,Trattoria/Osteria,Food,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant


#### Cluster 2

In this cluster we can find the most part of the cities with a wide offer of leaisure services and gourmet food.

In [0]:
pop_merged.loc[pop_merged['Cluster Labels'] == 1, pop_merged.columns[[0] + list(range(5, pop_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AGGIUS,1,History Museum,Italian Restaurant,Bar,Lake,Pizza Place,Food,Dessert Shop,Diner,Discount Store,Dive Bar
1,AGLIENTU,1,Gourmet Shop,Bed & Breakfast,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm
3,ALGHERO,1,Italian Restaurant,Hotel,Pizza Place,Restaurant,Café,Trattoria/Osteria,Seafood Restaurant,Beach Bar,Ice Cream Shop,Supermarket
5,ARDARA,1,Café,Restaurant,Trattoria/Osteria,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm
6,ARZACHENA,1,Italian Restaurant,Hotel,Café,Shopping Mall,Hardware Store,Flower Shop,Dessert Shop,Diner,Discount Store,Dive Bar
7,BADESI,1,Restaurant,Transportation Service,Ice Cream Shop,Pizza Place,Trattoria/Osteria,Flower Shop,Department Store,Dessert Shop,Diner,Discount Store
8,BANARI,1,Rental Car Location,Bed & Breakfast,Trattoria/Osteria,Food,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm
10,BERCHIDDA,1,Pharmacy,Jazz Club,Business Service,Flower Shop,Food,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store
11,BESSUDE,1,Scenic Lookout,Trattoria/Osteria,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm
12,BONNANARO,1,Plaza,Mountain,Restaurant,Financial or Legal Service,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar


#### Cluster 3

In this cluster main activities regard small and medium enterprises and related services.

In [0]:
pop_merged.loc[pop_merged['Cluster Labels'] == 2, pop_merged.columns[[0] + list(range(5, pop_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,ALÀ DEI SARDI,2,Construction & Landscaping,Trattoria/Osteria,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant
9,BENETUTTI,2,Construction & Landscaping,Pizza Place,Flower Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store
14,BONORVA,2,Construction & Landscaping,Residential Building (Apartment / Condo),Bakery,Farm,Food,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store
18,BUDDUSÒ,2,Construction & Landscaping,Hotel,Jewelry Store,Butcher,Financial or Legal Service,Food,Diner,Discount Store,Dive Bar,Electronics Store
21,BULZI,2,Construction & Landscaping,Historic Site,Bar,Non-Profit,Trattoria/Osteria,Food,Diner,Discount Store,Dive Bar,Electronics Store
37,ITTIRI,2,Pharmacy,Construction & Landscaping,Furniture / Home Store,Pizza Place,Flower Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar
41,LUOGOSANTO,2,Construction & Landscaping,Café,Restaurant,Food,Hardware Store,Harbor / Marina,Dessert Shop,Diner,Discount Store,Dive Bar
51,NULVI,2,Agriturismo,Construction & Landscaping,Pizza Place,Trattoria/Osteria,Food,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store
53,OLMEDO,2,Construction & Landscaping,Seafood Restaurant,Pizza Place,Trattoria/Osteria,Flower Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar
55,OSILO,2,Construction & Landscaping,Spa,Home Service,Art Museum,Arts & Crafts Store,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store


#### Cluster 4

These cities are characterized by agricoltural economy and genuine country food.

In [0]:
pop_merged.loc[pop_merged['Cluster Labels'] == 3, pop_merged.columns[[0] + list(range(5, pop_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,GIAVE,3,Italian Restaurant,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant
39,LAERRU,3,Italian Restaurant,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant
61,PATTADA,3,Italian Restaurant,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant


#### Cluster 5

In these municipality the activities are linked to every day life and the most common venue are bar and caffè as social meeting point of the few citizens.

In [0]:
pop_merged.loc[pop_merged['Cluster Labels'] == 4, pop_merged.columns[[0] + list(range(5, pop_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,ANELA,4,Bar,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant
20,BULTEI,4,Café,Bar,Trattoria/Osteria,Food Court,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant
30,ERULA,4,Bar,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant
76,SILIGO,4,Bar,Trattoria/Osteria,Food Court,Dessert Shop,Diner,Discount Store,Dive Bar,Electronics Store,Farm,Fast Food Restaurant


## Clustering tourist accommodation capacity

The following part of the analysis investigated the accomodation capacity according to the official regional classification. 

Data are provided by the regional administration in its opendata web portal and are available in various format, we used .csv.

In [0]:
acc_cap_2019 = pd.read_csv('http://opendata.sardegnaturismocloud.it/IT/turismo/offerta/ricettivita/capacita-strutture-ricettive/2019/capacita_ricettiva_2019.csv')

We translated columns name in english and set to 0 the non available data.

In [0]:
acc_cap_2019.rename(columns={'anno':'Year','Provincia':'Province','comune':'City','mese':'Month','Tipologia':'Type','Stelle':'Class','numero_strutture':'Nr','letti':'Beds','camere':'Rooms'}, inplace=True)
acc_cap_2019["Nr"] = acc_cap_2019["Nr"].fillna(0)
acc_cap_2019["Beds"] = acc_cap_2019["Beds"].fillna(0)
acc_cap_2019["Rooms"] = acc_cap_2019["Rooms"].fillna(0)
acc_cap_2019.head(3)

,Year,Province,City,Month,Type,Class,Nr,Beds,Rooms
0,2019,CittàmetropolitanaCagliari,ASSEMINI,1,Affittacamere,IICategoria,1,9.0,4.0
1,2019,CittàmetropolitanaCagliari,CAGLIARI,1,Affittacamere,NaN,8,57.0,29.0
2,2019,CittàmetropolitanaCagliari,CAGLIARI,1,Affittacamere,ICategoria,3,28.0,13.0


After that, we checked the unique values in the dataframe and replaced Nan values.

In [0]:
acc_cap_2019.Type.unique()

array(['Affittacamere', 'Agriturismo', 'Albergo', 'Albergoresidenziale',
       'Alloggiprivati', 'Bedandbreakfast', 'Caseeappartamentipervacanze',
       'Caseperferie', 'Ostelliperlagioventù', 'Residence', 'Campeggio',
       'BoatandBreakfast', 'Locanda', 'TurismoRurale',
       'Villaggituristici', 'Villaggioalbergo', 'Albergodiffuso'],
      dtype=object)

In [0]:
acc_cap_2019.Province.unique()

array(['CittàmetropolitanaCagliari', 'ProvinciadelSudSardegna',
       'ProvinciadiNuoro', 'ProvinciadiOristano', 'ProvinciadiSassari'],
      dtype=object)

In [0]:
acc_cap_2019.Class.unique()

array(['IICategoria', nan, 'ICategoria', '3Stelle', '4Stelle', '1Stella',
       '2Stelle', 'IIICategoria', '5Stelle', '5StelleLusso'], dtype=object)

In [0]:
acc_cap_2019["Class"].replace(np.nan, "No_class", inplace=True)
acc_cap_2019.Class.unique()

array(['IICategoria', 'No_class', 'ICategoria', '3Stelle', '4Stelle',
       '1Stella', '2Stelle', 'IIICategoria', '5Stelle', '5StelleLusso'],
      dtype=object)

In [0]:
acc_cap_2019 = acc_cap_2019[acc_cap_2019.Province == 'ProvinciadiSassari'].reset_index(drop=True)
acc_cap_2019.head(3)

,Year,Province,City,Month,Type,Class,Nr,Beds,Rooms
0,2019,ProvinciadiSassari,ALGHERO,1,Affittacamere,No_class,27,194.0,96.0
1,2019,ProvinciadiSassari,ALGHERO,1,Affittacamere,ICategoria,4,37.0,18.0
2,2019,ProvinciadiSassari,ALGHERO,1,Affittacamere,IICategoria,46,321.0,156.0


In [0]:
acc_cap_2019.dtypes

Year          int64
Province     object
City         object
Month         int64
Type         object
Class        object
Nr            int64
Beds        float64
Rooms       float64
dtype: object

In [0]:
acc_cap_2019[["Beds"]] = acc_cap_2019[["Beds"]].astype("int")
acc_cap_2019[["Rooms"]] = acc_cap_2019[["Rooms"]].astype("int")
acc_cap_2019.dtypes

Year         int64
Province    object
City        object
Month        int64
Type        object
Class       object
Nr           int64
Beds         int64
Rooms        int64
dtype: object

We merged Type and Class columns to make the analysis easier and set the number of available beds as key indicators of the accomodation capacity.

In [0]:
acc_cap_2019["Typology"] = acc_cap_2019["Type"] + " " + acc_cap_2019["Class"]
acc_cap_2019.drop(['Type', 'Class','Month', 'Nr', 'Rooms',"Year", "Province"],axis=1).reset_index(drop=True)
acc_cap_2019 = acc_cap_2019[["City","Typology","Beds"]]
acc_cap_2019 = acc_cap_2019.groupby(['City','Typology'],as_index=False).mean()
acc_cap_2019.head(3)

,City,Typology,Beds
0,AGGIUS,Affittacamere IICategoria,11.0
1,AGGIUS,Agriturismo No_class,38.0
2,AGGIUS,Alloggiprivati No_class,10.0


Then we prepared data for the cluster analysis.

In [0]:
pivot_acc_cap_2019 = pd.pivot_table(acc_cap_2019, values='Beds', index=['City'],
                    columns=['Typology'])
#, aggfunc=np.sum)
pivot_acc_cap_2019 = pivot_acc_cap_2019.fillna(0)
pivot_acc_cap_2019.head(3)

Typology,Affittacamere ICategoria,Affittacamere IICategoria,Affittacamere IIICategoria,Affittacamere No_class,Agriturismo No_class,Albergo 1Stella,Albergo 2Stelle,Albergo 3Stelle,Albergo 4Stelle,Albergo 5Stelle,Albergo 5StelleLusso,Albergodiffuso 3Stelle,Albergodiffuso 4Stelle,Albergodiffuso No_class,Albergoresidenziale 2Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 4Stelle,Alloggiprivati No_class,Bedandbreakfast 1Stella,Bedandbreakfast 2Stelle,Bedandbreakfast 3Stelle,Bedandbreakfast No_class,BoatandBreakfast No_class,Campeggio 1Stella,Campeggio 2Stelle,Campeggio 3Stelle,Campeggio 4Stelle,Caseeappartamentipervacanze ICategoria,Caseeappartamentipervacanze IICategoria,Caseeappartamentipervacanze IIICategoria,Caseeappartamentipervacanze No_class,Caseperferie IIICategoria,Locanda ICategoria,Locanda IICategoria,Locanda No_class,Ostelliperlagioventù IIICategoria,Residence ICategoria,Residence IICategoria,Residence IIICategoria,Residence No_class,TurismoRurale No_class,Villaggioalbergo 2Stelle,Villaggituristici 2Stelle,Villaggituristici 3Stelle,Villaggituristici 4Stelle
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AGGIUS,0.0,11.000000,0.000000,0.000000,38.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,10.000000,0.000000,0.0,0.000000,32.583333,0.0,0.0,0.0,0.0,0.0,0.0,8.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
AGLIENTU,0.0,0.000000,0.000000,0.000000,52.166667,0.0,23.0,747.857143,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,69.833333,0.000000,0.0,0.000000,9.750000,0.0,0.0,0.0,0.0,3523.5,0.0,346.916667,0.000000,0.000000,10.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
ALGHERO,48.0,466.333333,46.333333,203.833333,250.333333,0.0,0.0,976.750000,2465.25,536.0,0.0,0.0,0.0,0.0,275.0,264.083333,42.0,2081.333333,34.083333,47.0,114.083333,923.833333,4.0,0.0,0.0,2973.0,0.0,8.0,850.000000,97.583333,107.833333,74.583333,10.0,0.0,0.0,130.0,0.0,0.0,0.0,0.0,200.583333,0.0,0.0,200.0,0.0


In [0]:
pivot_acc_cap_2019 = pivot_acc_cap_2019.reset_index().rename_axis(None, axis=1)
pivot_acc_cap_2019.head(3)

,City,Affittacamere ICategoria,Affittacamere IICategoria,Affittacamere IIICategoria,Affittacamere No_class,Agriturismo No_class,Albergo 1Stella,Albergo 2Stelle,Albergo 3Stelle,Albergo 4Stelle,Albergo 5Stelle,Albergo 5StelleLusso,Albergodiffuso 3Stelle,Albergodiffuso 4Stelle,Albergodiffuso No_class,Albergoresidenziale 2Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 4Stelle,Alloggiprivati No_class,Bedandbreakfast 1Stella,Bedandbreakfast 2Stelle,Bedandbreakfast 3Stelle,Bedandbreakfast No_class,BoatandBreakfast No_class,Campeggio 1Stella,Campeggio 2Stelle,Campeggio 3Stelle,Campeggio 4Stelle,Caseeappartamentipervacanze ICategoria,Caseeappartamentipervacanze IICategoria,Caseeappartamentipervacanze IIICategoria,Caseeappartamentipervacanze No_class,Caseperferie IIICategoria,Locanda ICategoria,Locanda IICategoria,Locanda No_class,Ostelliperlagioventù IIICategoria,Residence ICategoria,Residence IICategoria,Residence IIICategoria,Residence No_class,TurismoRurale No_class,Villaggioalbergo 2Stelle,Villaggituristici 2Stelle,Villaggituristici 3Stelle,Villaggituristici 4Stelle
0,AGGIUS,0.0,11.000000,0.000000,0.000000,38.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,10.000000,0.000000,0.0,0.000000,32.583333,0.0,0.0,0.0,0.0,0.0,0.0,8.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,AGLIENTU,0.0,0.000000,0.000000,0.000000,52.166667,0.0,23.0,747.857143,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,69.833333,0.000000,0.0,0.000000,9.750000,0.0,0.0,0.0,0.0,3523.5,0.0,346.916667,0.000000,0.000000,10.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,ALGHERO,48.0,466.333333,46.333333,203.833333,250.333333,0.0,0.0,976.750000,2465.25,536.0,0.0,0.0,0.0,0.0,275.0,264.083333,42.0,2081.333333,34.083333,47.0,114.083333,923.833333,4.0,0.0,0.0,2973.0,0.0,8.0,850.000000,97.583333,107.833333,74.583333,10.0,0.0,0.0,130.0,0.0,0.0,0.0,0.0,200.583333,0.0,0.0,200.0,0.0


In [0]:
pivot_acc_cap_2019.sort_values(by=['City'], inplace=True)
pivot_acc_cap_2019.head(3)

,City,Affittacamere ICategoria,Affittacamere IICategoria,Affittacamere IIICategoria,Affittacamere No_class,Agriturismo No_class,Albergo 1Stella,Albergo 2Stelle,Albergo 3Stelle,Albergo 4Stelle,Albergo 5Stelle,Albergo 5StelleLusso,Albergodiffuso 3Stelle,Albergodiffuso 4Stelle,Albergodiffuso No_class,Albergoresidenziale 2Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 4Stelle,Alloggiprivati No_class,Bedandbreakfast 1Stella,Bedandbreakfast 2Stelle,Bedandbreakfast 3Stelle,Bedandbreakfast No_class,BoatandBreakfast No_class,Campeggio 1Stella,Campeggio 2Stelle,Campeggio 3Stelle,Campeggio 4Stelle,Caseeappartamentipervacanze ICategoria,Caseeappartamentipervacanze IICategoria,Caseeappartamentipervacanze IIICategoria,Caseeappartamentipervacanze No_class,Caseperferie IIICategoria,Locanda ICategoria,Locanda IICategoria,Locanda No_class,Ostelliperlagioventù IIICategoria,Residence ICategoria,Residence IICategoria,Residence IIICategoria,Residence No_class,TurismoRurale No_class,Villaggioalbergo 2Stelle,Villaggituristici 2Stelle,Villaggituristici 3Stelle,Villaggituristici 4Stelle
0,AGGIUS,0.0,11.000000,0.000000,0.000000,38.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,10.000000,0.000000,0.0,0.000000,32.583333,0.0,0.0,0.0,0.0,0.0,0.0,8.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,AGLIENTU,0.0,0.000000,0.000000,0.000000,52.166667,0.0,23.0,747.857143,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,69.833333,0.000000,0.0,0.000000,9.750000,0.0,0.0,0.0,0.0,3523.5,0.0,346.916667,0.000000,0.000000,10.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,ALGHERO,48.0,466.333333,46.333333,203.833333,250.333333,0.0,0.0,976.750000,2465.25,536.0,0.0,0.0,0.0,0.0,275.0,264.083333,42.0,2081.333333,34.083333,47.0,114.083333,923.833333,4.0,0.0,0.0,2973.0,0.0,8.0,850.000000,97.583333,107.833333,74.583333,10.0,0.0,0.0,130.0,0.0,0.0,0.0,0.0,200.583333,0.0,0.0,200.0,0.0


In [0]:
pivot_acc_cap_2019.sum()==0

City                                        False
Affittacamere ICategoria                    False
Affittacamere IICategoria                   False
Affittacamere IIICategoria                  False
Affittacamere No_class                      False
Agriturismo No_class                        False
Albergo 1Stella                             False
Albergo 2Stelle                             False
Albergo 3Stelle                             False
Albergo 4Stelle                             False
Albergo 5Stelle                             False
Albergo 5StelleLusso                        False
Albergodiffuso 3Stelle                      False
Albergodiffuso 4Stelle                      False
Albergodiffuso No_class                     False
Albergoresidenziale 2Stelle                 False
Albergoresidenziale 3Stelle                 False
Albergoresidenziale 4Stelle                 False
Alloggiprivati No_class                     False
Bedandbreakfast 1Stella                     False


We dropped the column "Locanda II categoria" because it was null.

In [0]:
pivot_acc_cap_2019.drop('Locanda IICategoria',axis=1,inplace=True)
pivot_acc_cap_2019.head(3)

,City,Affittacamere ICategoria,Affittacamere IICategoria,Affittacamere IIICategoria,Affittacamere No_class,Agriturismo No_class,Albergo 1Stella,Albergo 2Stelle,Albergo 3Stelle,Albergo 4Stelle,Albergo 5Stelle,Albergo 5StelleLusso,Albergodiffuso 3Stelle,Albergodiffuso 4Stelle,Albergodiffuso No_class,Albergoresidenziale 2Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 4Stelle,Alloggiprivati No_class,Bedandbreakfast 1Stella,Bedandbreakfast 2Stelle,Bedandbreakfast 3Stelle,Bedandbreakfast No_class,BoatandBreakfast No_class,Campeggio 1Stella,Campeggio 2Stelle,Campeggio 3Stelle,Campeggio 4Stelle,Caseeappartamentipervacanze ICategoria,Caseeappartamentipervacanze IICategoria,Caseeappartamentipervacanze IIICategoria,Caseeappartamentipervacanze No_class,Caseperferie IIICategoria,Locanda ICategoria,Locanda No_class,Ostelliperlagioventù IIICategoria,Residence ICategoria,Residence IICategoria,Residence IIICategoria,Residence No_class,TurismoRurale No_class,Villaggioalbergo 2Stelle,Villaggituristici 2Stelle,Villaggituristici 3Stelle,Villaggituristici 4Stelle
0,AGGIUS,0.0,11.000000,0.000000,0.000000,38.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,10.000000,0.000000,0.0,0.000000,32.583333,0.0,0.0,0.0,0.0,0.0,0.0,8.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,AGLIENTU,0.0,0.000000,0.000000,0.000000,52.166667,0.0,23.0,747.857143,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,69.833333,0.000000,0.0,0.000000,9.750000,0.0,0.0,0.0,0.0,3523.5,0.0,346.916667,0.000000,0.000000,10.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,ALGHERO,48.0,466.333333,46.333333,203.833333,250.333333,0.0,0.0,976.750000,2465.25,536.0,0.0,0.0,0.0,0.0,275.0,264.083333,42.0,2081.333333,34.083333,47.0,114.083333,923.833333,4.0,0.0,0.0,2973.0,0.0,8.0,850.000000,97.583333,107.833333,74.583333,10.0,0.0,130.0,0.0,0.0,0.0,0.0,200.583333,0.0,0.0,200.0,0.0


To preserve interoperability we checked the spelling of the municipalities names within this dataset and the previous one.

In [0]:
Test1 = pd.concat([pop['City'],pivot_acc_cap_2019['City']], axis=1, sort=False)
Test1

,City,City
0,AGGIUS,AGGIUS
1,AGLIENTU,AGLIENTU
2,ALÀ DEI SARDI,ALGHERO
3,ALGHERO,ALÀDEISARDI
4,ANELA,ANELA
5,ARDARA,ARDARA
6,ARZACHENA,ARZACHENA
7,BADESI,BADESI
8,BANARI,BANARI
9,BENETUTTI,BENETUTTI


In [0]:
pivot_acc_cap_2019.City.replace(['SASSARI'],'SASSARI, SS', inplace=True)
pivot_acc_cap_2019.City.replace(['ALÀDEISARDI'],'ALÀ DEI SARDI', inplace=True)
pivot_acc_cap_2019.City.replace(['LAMADDALENA'],'LA MADDALENA', inplace=True)
pivot_acc_cap_2019.City.replace(['LOIRIPORTOSANPAOLO'],'LOIRI PORTO SAN PAOLO', inplace=True)
pivot_acc_cap_2019.City.replace(['MONTELEONEROCCADORIA'],'MONTELEONE ROCCA DORIA', inplace=True)
pivot_acc_cap_2019.City.replace(['NUGHEDUSANNICOLO'],'NUGHEDU SAN NICOLÒ', inplace=True)
pivot_acc_cap_2019.City.replace(['PORTOTORRES'],'PORTO TORRES', inplace=True)
pivot_acc_cap_2019.City.replace(["SANT'ANTONIODIGALLURA"],"SANT'ANTONIO DI GALLURA", inplace=True)
pivot_acc_cap_2019.City.replace(['SANTAMARIACOGHINAS'],'SANTA MARIA COGHINAS', inplace=True)
pivot_acc_cap_2019.City.replace(['SANTATERESAGALLURA'],'SANTA TERESA GALLURA', inplace=True)
pivot_acc_cap_2019.City.replace(['SANTEODORO'],'SAN TEODORO', inplace=True)
pivot_acc_cap_2019.City.replace(['TEMPIOPAUSANIA'],'TEMPIO PAUSANIA', inplace=True)
pivot_acc_cap_2019.City.replace(["TRINITÀD'AGULTUEVIGNOLA"],"TRINITÀ D'AGULTU E VIGNOLA", inplace=True)
pivot_acc_cap_2019.City.replace(['VILLANOVAMONTELEONE'],'VILLANOVA MONTELEONE', inplace=True)
Test2 = pd.concat([pop['City'],pivot_acc_cap_2019['City']], axis=1, sort=False)
Test2

,City,City
0,AGGIUS,AGGIUS
1,AGLIENTU,AGLIENTU
2,ALÀ DEI SARDI,ALGHERO
3,ALGHERO,ALÀ DEI SARDI
4,ANELA,ANELA
5,ARDARA,ARDARA
6,ARZACHENA,ARZACHENA
7,BADESI,BADESI
8,BANARI,BANARI
9,BENETUTTI,BENETUTTI


In [0]:
pivot_acc_cap_2019.shape

(84, 45)

and from this point forward, the clustering process has been the same as in the first analysis.

In [0]:
TCA = pivot_acc_cap_2019.iloc[0:,1:].apply(lambda x: x / x.sum())
TCA.head(3)

,Affittacamere ICategoria,Affittacamere IICategoria,Affittacamere IIICategoria,Affittacamere No_class,Agriturismo No_class,Albergo 1Stella,Albergo 2Stelle,Albergo 3Stelle,Albergo 4Stelle,Albergo 5Stelle,Albergo 5StelleLusso,Albergodiffuso 3Stelle,Albergodiffuso 4Stelle,Albergodiffuso No_class,Albergoresidenziale 2Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 4Stelle,Alloggiprivati No_class,Bedandbreakfast 1Stella,Bedandbreakfast 2Stelle,Bedandbreakfast 3Stelle,Bedandbreakfast No_class,BoatandBreakfast No_class,Campeggio 1Stella,Campeggio 2Stelle,Campeggio 3Stelle,Campeggio 4Stelle,Caseeappartamentipervacanze ICategoria,Caseeappartamentipervacanze IICategoria,Caseeappartamentipervacanze IIICategoria,Caseeappartamentipervacanze No_class,Caseperferie IIICategoria,Locanda ICategoria,Locanda No_class,Ostelliperlagioventù IIICategoria,Residence ICategoria,Residence IICategoria,Residence IIICategoria,Residence No_class,TurismoRurale No_class,Villaggioalbergo 2Stelle,Villaggituristici 2Stelle,Villaggituristici 3Stelle,Villaggituristici 4Stelle
0,0.000000,0.010080,0.000000,0.000000,0.018417,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.001070,0.000000,0.00000,0.00000,0.008931,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.001182,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,0.000000,0.000000,0.000000,0.000000,0.025284,0.0,0.038097,0.068337,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.007474,0.000000,0.00000,0.00000,0.002673,0.0,0.0,0.0,0.000000,0.39479,0.000000,0.051258,0.000000,0.000000,0.075424,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,0.187867,0.427345,0.496429,0.298075,0.121329,0.0,0.000000,0.089253,0.116002,0.129555,0.0,0.0,0.0,0.0,0.846154,0.041359,0.026614,0.222767,0.367475,0.25522,0.42875,0.253229,1.0,0.0,0.0,0.228995,0.00000,0.005908,0.125590,0.171625,0.086853,0.562539,0.47619,0.0,0.472727,0.0,0.0,0.0,0.0,0.359576,0.0,0.0,0.468384,0.0


In [0]:
TCA2 = pd.concat([pivot_acc_cap_2019['City'], TCA], axis=1, sort=False)
TCA2

,City,Affittacamere ICategoria,Affittacamere IICategoria,Affittacamere IIICategoria,Affittacamere No_class,Agriturismo No_class,Albergo 1Stella,Albergo 2Stelle,Albergo 3Stelle,Albergo 4Stelle,Albergo 5Stelle,Albergo 5StelleLusso,Albergodiffuso 3Stelle,Albergodiffuso 4Stelle,Albergodiffuso No_class,Albergoresidenziale 2Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 4Stelle,Alloggiprivati No_class,Bedandbreakfast 1Stella,Bedandbreakfast 2Stelle,Bedandbreakfast 3Stelle,Bedandbreakfast No_class,BoatandBreakfast No_class,Campeggio 1Stella,Campeggio 2Stelle,Campeggio 3Stelle,Campeggio 4Stelle,Caseeappartamentipervacanze ICategoria,Caseeappartamentipervacanze IICategoria,Caseeappartamentipervacanze IIICategoria,Caseeappartamentipervacanze No_class,Caseperferie IIICategoria,Locanda ICategoria,Locanda No_class,Ostelliperlagioventù IIICategoria,Residence ICategoria,Residence IICategoria,Residence IIICategoria,Residence No_class,TurismoRurale No_class,Villaggioalbergo 2Stelle,Villaggituristici 2Stelle,Villaggituristici 3Stelle,Villaggituristici 4Stelle
0,AGGIUS,0.000000,0.010080,0.000000,0.000000,0.018417,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.001070,0.000000,0.000000,0.000000,0.008931,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.001182,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,AGLIENTU,0.000000,0.000000,0.000000,0.000000,0.025284,0.000000,0.038097,0.068337,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.007474,0.000000,0.000000,0.000000,0.002673,0.0,0.0,0.000000,0.000000,0.394790,0.000000,0.051258,0.000000,0.000000,0.075424,0.00000,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,ALGHERO,0.187867,0.427345,0.496429,0.298075,0.121329,0.000000,0.000000,0.089253,0.116002,0.129555,0.0,0.0,0.0,0.0,0.846154,0.041359,0.026614,0.222767,0.367475,0.255220,0.428750,0.253229,1.0,0.0,0.000000,0.228995,0.000000,0.005908,0.125590,0.171625,0.086853,0.562539,0.47619,0.0000,0.472727,0.0,0.000000,0.000000,0.0,0.359576,0.0,0.0,0.468384,0.000000
3,ALÀ DEI SARDI,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000428,0.000000,0.010860,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,ANELA,0.000000,0.000000,0.000000,0.000000,0.002908,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000428,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
5,ARDARA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.021721,0.000000,0.001645,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
6,ARZACHENA,0.091977,0.019932,0.042857,0.029978,0.067167,0.000000,0.033128,0.079582,0.157234,0.214173,1.0,0.0,0.0,0.0,0.000000,0.020986,0.614729,0.067858,0.043127,0.000000,0.037582,0.030928,0.0,0.0,0.000000,0.019949,0.325378,0.085332,0.036889,0.000000,0.035036,0.000000,0.00000,0.0000,0.000000,1.0,0.296254,0.000000,0.0,0.087840,0.0,0.0,0.000000,0.000000
7,BADESI,0.000000,0.007331,0.000000,0.000000,0.002908,0.000000,0.019877,0.007858,0.057752,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.007091,0.000000,0.000000,0.000000,0.001645,0.0,0.0,0.000000,0.077025,0.000000,0.089845,0.049953,0.307783,0.000000,0.000000,0.00000,0.0000,0.000000,0.0,0.151168,0.000000,0.0,0.035853,0.0,0.0,0.000000,0.000000
8,BANARI,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0

In [0]:
num_top_accom = 10

for accom in TCA2['City']:
    print("----"+accom+"----")
    temp = TCA2[TCA2['City'] == accom].T.reset_index()
    temp.columns = ['Accommodation','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_accom))
    print('\n')

----AGGIUS----
                              Accommodation  freq
0                      Agriturismo No_class  0.02
1                 Affittacamere IICategoria  0.01
2                  Bedandbreakfast No_class  0.01
3                          Locanda No_class  0.00
4                         Campeggio 3Stelle  0.00
5                         Campeggio 4Stelle  0.00
6    Caseeappartamentipervacanze ICategoria  0.00
7   Caseeappartamentipervacanze IICategoria  0.00
8  Caseeappartamentipervacanze IIICategoria  0.00
9      Caseeappartamentipervacanze No_class  0.00


----AGLIENTU----
                             Accommodation  freq
0                        Campeggio 4Stelle  0.39
1                Caseperferie IIICategoria  0.08
2                          Albergo 3Stelle  0.07
3  Caseeappartamentipervacanze IICategoria  0.05
4                          Albergo 2Stelle  0.04
5                     Agriturismo No_class  0.03
6                  Alloggiprivati No_class  0.01
7                 Affitt

First, let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_accom(row, num_top_accom):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_accom]

Now let's create the new dataframe and display the top 10 accommodation for each city.

In [0]:
num_top_accom = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top accommodations
columns = ['City']
for ind in np.arange(num_top_accom):
    try:
        columns.append('{}{} Most Common Accommodation'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Accommodation'.format(ind+1))

# create a new dataframe
accom_sorted = pd.DataFrame(columns=columns)
accom_sorted['City'] = TCA2['City']

for ind in np.arange(TCA2.shape[0]):
    accom_sorted.iloc[ind, 1:] = return_most_common_accom(TCA2.iloc[ind, :], num_top_accom)

accom_sorted

,City,1st Most Common Accommodation,2nd Most Common Accommodation,3rd Most Common Accommodation,4th Most Common Accommodation,5th Most Common Accommodation,6th Most Common Accommodation,7th Most Common Accommodation,8th Most Common Accommodation,9th Most Common Accommodation,10th Most Common Accommodation
0,AGGIUS,Agriturismo No_class,Affittacamere IICategoria,Bedandbreakfast No_class,Caseeappartamentipervacanze IICategoria,Alloggiprivati No_class,Albergodiffuso 3Stelle,Bedandbreakfast 1Stella,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle
1,AGLIENTU,Campeggio 4Stelle,Caseperferie IIICategoria,Albergo 3Stelle,Caseeappartamentipervacanze IICategoria,Albergo 2Stelle,Agriturismo No_class,Alloggiprivati No_class,Bedandbreakfast No_class,Albergodiffuso 4Stelle,Albergoresidenziale 4Stelle
2,ALGHERO,BoatandBreakfast No_class,Albergoresidenziale 2Stelle,Caseperferie IIICategoria,Affittacamere IIICategoria,Locanda ICategoria,Ostelliperlagioventù IIICategoria,Villaggituristici 3Stelle,Bedandbreakfast 3Stelle,Affittacamere IICategoria,Bedandbreakfast 1Stella
3,ALÀ DEI SARDI,Bedandbreakfast 2Stelle,Alloggiprivati No_class,Villaggituristici 4Stelle,Villaggituristici 3Stelle,Bedandbreakfast 1Stella,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class,Albergodiffuso 4Stelle
4,ANELA,Agriturismo No_class,Alloggiprivati No_class,Villaggituristici 4Stelle,Albergodiffuso 3Stelle,Bedandbreakfast 2Stelle,Bedandbreakfast 1Stella,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class
5,ARDARA,Bedandbreakfast 2Stelle,Bedandbreakfast No_class,Villaggituristici 3Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class,Albergodiffuso 4Stelle
6,ARZACHENA,Albergo 5StelleLusso,Residence ICategoria,Albergoresidenziale 4Stelle,Campeggio 4Stelle,Residence IICategoria,Albergo 5Stelle,Albergo 4Stelle,Affittacamere ICategoria,TurismoRurale No_class,Caseeappartamentipervacanze ICategoria
7,BADESI,Caseeappartamentipervacanze IIICategoria,Residence IICategoria,Caseeappartamentipervacanze ICategoria,Campeggio 3Stelle,Albergo 4Stelle,Caseeappartamentipervacanze IICategoria,TurismoRurale No_class,Albergo 2Stelle,Albergo 3Stelle,Affittacamere IICategoria
8,BANARI,Bedandbreakfast No_class,Villaggituristici 3Stelle,Bedandbreakfast 2Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class,Albergodiffuso 4Stelle
9,BENETUTTI,Bedandbreakfast 2Stelle,Albergo 3Stelle,Bedandbreakfast No_class,Albergo 4Stelle,Agriturismo No_class,Albergodiffuso 4Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle


Run *k*-means to cluster the cities into 5 clusters.

In [0]:
# set number of clusters
kclusters = 5

TCA2_clustering = TCA2.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TCA2_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 3, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 accomodation facilities for each city.

In [0]:
# add clustering labels
accom_sorted.insert(0, 'Cluster Acc_Labels', kmeans.labels_)

accom_merged = pop

# merge pop_grouped with pop to add latitude/longitude for each city
accom_merged = accom_merged.join(accom_sorted.set_index('City'), on='City')
accom_merged.dropna(subset=['Cluster Acc_Labels'],inplace=True)
accom_merged[["Cluster Acc_Labels"]] = accom_merged[["Cluster Acc_Labels"]].astype("int")
accom_merged.head(90) # check the last columns!

,City,Province,Population,Longitude,Latitude,Cluster Acc_Labels,1st Most Common Accommodation,2nd Most Common Accommodation,3rd Most Common Accommodation,4th Most Common Accommodation,5th Most Common Accommodation,6th Most Common Accommodation,7th Most Common Accommodation,8th Most Common Accommodation,9th Most Common Accommodation,10th Most Common Accommodation
0,AGGIUS,SASSARI,1436,9.065222,40.929345,0,Agriturismo No_class,Affittacamere IICategoria,Bedandbreakfast No_class,Caseeappartamentipervacanze IICategoria,Alloggiprivati No_class,Albergodiffuso 3Stelle,Bedandbreakfast 1Stella,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle
1,AGLIENTU,SASSARI,1221,9.112719,41.079573,0,Campeggio 4Stelle,Caseperferie IIICategoria,Albergo 3Stelle,Caseeappartamentipervacanze IICategoria,Albergo 2Stelle,Agriturismo No_class,Alloggiprivati No_class,Bedandbreakfast No_class,Albergodiffuso 4Stelle,Albergoresidenziale 4Stelle
2,ALÀ DEI SARDI,SASSARI,1846,9.327829,40.650066,0,Bedandbreakfast 2Stelle,Alloggiprivati No_class,Villaggituristici 4Stelle,Villaggituristici 3Stelle,Bedandbreakfast 1Stella,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class,Albergodiffuso 4Stelle
3,ALGHERO,SASSARI,43845,8.315147,40.560021,2,BoatandBreakfast No_class,Albergoresidenziale 2Stelle,Caseperferie IIICategoria,Affittacamere IIICategoria,Locanda ICategoria,Ostelliperlagioventù IIICategoria,Villaggituristici 3Stelle,Bedandbreakfast 3Stelle,Affittacamere IICategoria,Bedandbreakfast 1Stella
4,ANELA,SASSARI,615,9.057427,40.441760,0,Agriturismo No_class,Alloggiprivati No_class,Villaggituristici 4Stelle,Albergodiffuso 3Stelle,Bedandbreakfast 2Stelle,Bedandbreakfast 1Stella,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class
5,ARDARA,SASSARI,767,8.810319,40.621761,0,Bedandbreakfast 2Stelle,Bedandbreakfast No_class,Villaggituristici 3Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class,Albergodiffuso 4Stelle
6,ARZACHENA,SASSARI,13816,9.387612,41.079898,3,Albergo 5StelleLusso,Residence ICategoria,Albergoresidenziale 4Stelle,Campeggio 4Stelle,Residence IICategoria,Albergo 5Stelle,Albergo 4Stelle,Affittacamere ICategoria,TurismoRurale No_class,Caseeappartamentipervacanze ICategoria
7,BADESI,SASSARI,1 862,8.884216,40.965069,0,Caseeappartamentipervacanze IIICategoria,Residence IICategoria,Caseeappartamentipervacanze ICategoria,Campeggio 3Stelle,Albergo 4Stelle,Caseeappartamentipervacanze IICategoria,TurismoRurale No_class,Albergo 2Stelle,Albergo 3Stelle,Affittacamere IICategoria
8,BANARI,SASSARI,550,8.697371,40.572927,0,Bedandbreakfast No_class,Villaggituristici 3Stelle,Bedandbreakfast 2Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class,Albergodiffuso 4Stelle
9,BENETUTTI,SASSARI,1793,9.168829,40.456461,0,Bedandbreakfast 2Stelle,Albergo 3Stelle,Bedandbreakfast No_class,Albergo 4Stelle,Agriturismo No_class,Albergodiffuso 4Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle


Finally, let's visualize the resulting clusters.

In [0]:
accom_merged[['Cluster Acc_Labels']] = accom_merged[['Cluster Acc_Labels']].astype("int")
accom_merged.dtypes

City                               object
Province                           object
Population                         object
Longitude                         float64
Latitude                          float64
Cluster Acc_Labels                  int64
1st Most Common Accommodation      object
2nd Most Common Accommodation      object
3rd Most Common Accommodation      object
4th Most Common Accommodation      object
5th Most Common Accommodation      object
6th Most Common Accommodation      object
7th Most Common Accommodation      object
8th Most Common Accommodation      object
9th Most Common Accommodation      object
10th Most Common Accommodation     object
dtype: object

In [0]:
accom_merged.head(90)

,City,Province,Population,Longitude,Latitude,Cluster Acc_Labels,1st Most Common Accommodation,2nd Most Common Accommodation,3rd Most Common Accommodation,4th Most Common Accommodation,5th Most Common Accommodation,6th Most Common Accommodation,7th Most Common Accommodation,8th Most Common Accommodation,9th Most Common Accommodation,10th Most Common Accommodation
0,AGGIUS,SASSARI,1436,9.065222,40.929345,0,Agriturismo No_class,Affittacamere IICategoria,Bedandbreakfast No_class,Caseeappartamentipervacanze IICategoria,Alloggiprivati No_class,Albergodiffuso 3Stelle,Bedandbreakfast 1Stella,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle
1,AGLIENTU,SASSARI,1221,9.112719,41.079573,0,Campeggio 4Stelle,Caseperferie IIICategoria,Albergo 3Stelle,Caseeappartamentipervacanze IICategoria,Albergo 2Stelle,Agriturismo No_class,Alloggiprivati No_class,Bedandbreakfast No_class,Albergodiffuso 4Stelle,Albergoresidenziale 4Stelle
2,ALÀ DEI SARDI,SASSARI,1846,9.327829,40.650066,0,Bedandbreakfast 2Stelle,Alloggiprivati No_class,Villaggituristici 4Stelle,Villaggituristici 3Stelle,Bedandbreakfast 1Stella,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class,Albergodiffuso 4Stelle
3,ALGHERO,SASSARI,43845,8.315147,40.560021,2,BoatandBreakfast No_class,Albergoresidenziale 2Stelle,Caseperferie IIICategoria,Affittacamere IIICategoria,Locanda ICategoria,Ostelliperlagioventù IIICategoria,Villaggituristici 3Stelle,Bedandbreakfast 3Stelle,Affittacamere IICategoria,Bedandbreakfast 1Stella
4,ANELA,SASSARI,615,9.057427,40.441760,0,Agriturismo No_class,Alloggiprivati No_class,Villaggituristici 4Stelle,Albergodiffuso 3Stelle,Bedandbreakfast 2Stelle,Bedandbreakfast 1Stella,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class
5,ARDARA,SASSARI,767,8.810319,40.621761,0,Bedandbreakfast 2Stelle,Bedandbreakfast No_class,Villaggituristici 3Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class,Albergodiffuso 4Stelle
6,ARZACHENA,SASSARI,13816,9.387612,41.079898,3,Albergo 5StelleLusso,Residence ICategoria,Albergoresidenziale 4Stelle,Campeggio 4Stelle,Residence IICategoria,Albergo 5Stelle,Albergo 4Stelle,Affittacamere ICategoria,TurismoRurale No_class,Caseeappartamentipervacanze ICategoria
7,BADESI,SASSARI,1 862,8.884216,40.965069,0,Caseeappartamentipervacanze IIICategoria,Residence IICategoria,Caseeappartamentipervacanze ICategoria,Campeggio 3Stelle,Albergo 4Stelle,Caseeappartamentipervacanze IICategoria,TurismoRurale No_class,Albergo 2Stelle,Albergo 3Stelle,Affittacamere IICategoria
8,BANARI,SASSARI,550,8.697371,40.572927,0,Bedandbreakfast No_class,Villaggituristici 3Stelle,Bedandbreakfast 2Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class,Albergodiffuso 4Stelle
9,BENETUTTI,SASSARI,1793,9.168829,40.456461,0,Bedandbreakfast 2Stelle,Albergo 3Stelle,Bedandbreakfast No_class,Albergo 4Stelle,Agriturismo No_class,Albergodiffuso 4Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle


In [0]:
# create map
map_accom_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(accom_merged['Latitude'], accom_merged['Longitude'], accom_merged['City'], accom_merged['Cluster Acc_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_accom_clusters)
       
map_accom_clusters

## Examine Accomodation Clusters

As in the first case, the most part of the province show the same level of accomodation facilities and only the ancient city of Alghero in the West and the well known area of the Costa Smeralda in the North East show a sensible difference.

#### Cluster 1

This is the cluster more populated showing common characteristics all over the area.

In [0]:
accom_merged.loc[accom_merged['Cluster Acc_Labels'] == 0, accom_merged.columns[[0] + list(range(5, accom_merged.shape[1]))]]

,City,Cluster Acc_Labels,1st Most Common Accommodation,2nd Most Common Accommodation,3rd Most Common Accommodation,4th Most Common Accommodation,5th Most Common Accommodation,6th Most Common Accommodation,7th Most Common Accommodation,8th Most Common Accommodation,9th Most Common Accommodation,10th Most Common Accommodation
0,AGGIUS,0,Agriturismo No_class,Affittacamere IICategoria,Bedandbreakfast No_class,Caseeappartamentipervacanze IICategoria,Alloggiprivati No_class,Albergodiffuso 3Stelle,Bedandbreakfast 1Stella,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle
1,AGLIENTU,0,Campeggio 4Stelle,Caseperferie IIICategoria,Albergo 3Stelle,Caseeappartamentipervacanze IICategoria,Albergo 2Stelle,Agriturismo No_class,Alloggiprivati No_class,Bedandbreakfast No_class,Albergodiffuso 4Stelle,Albergoresidenziale 4Stelle
2,ALÀ DEI SARDI,0,Bedandbreakfast 2Stelle,Alloggiprivati No_class,Villaggituristici 4Stelle,Villaggituristici 3Stelle,Bedandbreakfast 1Stella,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class,Albergodiffuso 4Stelle
4,ANELA,0,Agriturismo No_class,Alloggiprivati No_class,Villaggituristici 4Stelle,Albergodiffuso 3Stelle,Bedandbreakfast 2Stelle,Bedandbreakfast 1Stella,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class
5,ARDARA,0,Bedandbreakfast 2Stelle,Bedandbreakfast No_class,Villaggituristici 3Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class,Albergodiffuso 4Stelle
7,BADESI,0,Caseeappartamentipervacanze IIICategoria,Residence IICategoria,Caseeappartamentipervacanze ICategoria,Campeggio 3Stelle,Albergo 4Stelle,Caseeappartamentipervacanze IICategoria,TurismoRurale No_class,Albergo 2Stelle,Albergo 3Stelle,Affittacamere IICategoria
8,BANARI,0,Bedandbreakfast No_class,Villaggituristici 3Stelle,Bedandbreakfast 2Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class,Albergodiffuso 4Stelle
9,BENETUTTI,0,Bedandbreakfast 2Stelle,Albergo 3Stelle,Bedandbreakfast No_class,Albergo 4Stelle,Agriturismo No_class,Albergodiffuso 4Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle
10,BERCHIDDA,0,Campeggio 2Stelle,Affittacamere ICategoria,Albergo 3Stelle,Agriturismo No_class,Bedandbreakfast No_class,Alloggiprivati No_class,Albergo 5Stelle,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle
11,BESSUDE,0,Agriturismo No_class,Bedandbreakfast No_class,Albergodiffuso 3Stelle,Bedandbreakfast 2Stelle,Bedandbreakfast 1Stella,Alloggiprivati No_class,Albergoresidenziale 4Stelle,Albergoresidenziale 3Stelle,Albergoresidenziale 2Stelle,Albergodiffuso No_class


#### Cluster 2

Olbia, Alghero, La Maddalena e Arzachena ("Costa Smeralda") are unique places that built up their touristic offer over decades. Most common facilities in Olbia are resorts, in Alghero are B&B and private holiday houses, in Arzachena are high level hotels and resorts and in La Maddalena medium level resorts.

In [0]:
accom_merged.loc[accom_merged['Cluster Acc_Labels'] == 1, accom_merged.columns[[0] + list(range(5, accom_merged.shape[1]))]]

,City,Cluster Acc_Labels,1st Most Common Accommodation,2nd Most Common Accommodation,3rd Most Common Accommodation,4th Most Common Accommodation,5th Most Common Accommodation,6th Most Common Accommodation,7th Most Common Accommodation,8th Most Common Accommodation,9th Most Common Accommodation,10th Most Common Accommodation
52,OLBIA,1,Villaggioalbergo 2Stelle,Albergodiffuso 3Stelle,Albergodiffuso 4Stelle,Locanda No_class,Caseeappartamentipervacanze ICategoria,Residence IIICategoria,Alloggiprivati No_class,Albergo 1Stella,Bedandbreakfast 1Stella,Affittacamere No_class


#### Cluster 3

In [0]:
accom_merged.loc[accom_merged['Cluster Acc_Labels'] == 2, accom_merged.columns[[0] + list(range(5, accom_merged.shape[1]))]]

,City,Cluster Acc_Labels,1st Most Common Accommodation,2nd Most Common Accommodation,3rd Most Common Accommodation,4th Most Common Accommodation,5th Most Common Accommodation,6th Most Common Accommodation,7th Most Common Accommodation,8th Most Common Accommodation,9th Most Common Accommodation,10th Most Common Accommodation
3,ALGHERO,2,BoatandBreakfast No_class,Albergoresidenziale 2Stelle,Caseperferie IIICategoria,Affittacamere IIICategoria,Locanda ICategoria,Ostelliperlagioventù IIICategoria,Villaggituristici 3Stelle,Bedandbreakfast 3Stelle,Affittacamere IICategoria,Bedandbreakfast 1Stella


#### Cluster 4

In [0]:
accom_merged.loc[accom_merged['Cluster Acc_Labels'] == 3, accom_merged.columns[[0] + list(range(5, accom_merged.shape[1]))]]

,City,Cluster Acc_Labels,1st Most Common Accommodation,2nd Most Common Accommodation,3rd Most Common Accommodation,4th Most Common Accommodation,5th Most Common Accommodation,6th Most Common Accommodation,7th Most Common Accommodation,8th Most Common Accommodation,9th Most Common Accommodation,10th Most Common Accommodation
6,ARZACHENA,3,Albergo 5StelleLusso,Residence ICategoria,Albergoresidenziale 4Stelle,Campeggio 4Stelle,Residence IICategoria,Albergo 5Stelle,Albergo 4Stelle,Affittacamere ICategoria,TurismoRurale No_class,Caseeappartamentipervacanze ICategoria


#### Cluster 5

In [0]:
accom_merged.loc[accom_merged['Cluster Acc_Labels'] == 4, accom_merged.columns[[0] + list(range(5, accom_merged.shape[1]))]]

,City,Cluster Acc_Labels,1st Most Common Accommodation,2nd Most Common Accommodation,3rd Most Common Accommodation,4th Most Common Accommodation,5th Most Common Accommodation,6th Most Common Accommodation,7th Most Common Accommodation,8th Most Common Accommodation,9th Most Common Accommodation,10th Most Common Accommodation
38,LA MADDALENA,4,Villaggituristici 2Stelle,Villaggituristici 3Stelle,Campeggio 2Stelle,Albergoresidenziale 2Stelle,Caseeappartamentipervacanze No_class,Affittacamere ICategoria,Albergo 3Stelle,Residence IICategoria,Albergo 5Stelle,Alloggiprivati No_class


## Cluster cities by tourist nationality

In this point too, data, regarding year 2019, have been provided by regional authorities on the opendata website. We conducted the analysis by tourists nationalities investigating if there are different  incoming channels to the various destination in the province.

In [0]:
mov_tur_com_2019 = pd.read_csv('http://opendata.sardegnaturismocloud.it/IT/turismo/domanda/ricettivita/movimenti-turistici/2019/movimenti_comuni_2019.csv')

In [0]:
mov_tur_com_2019.rename(columns={'anno':'Year','provincia':'Province','comune':'City','mese':'Month','provenienza':'From','arrivi':'Arrivals','presenze':'Presences'}, inplace=True)
mov_tur_com_2019["Arrivals"] = mov_tur_com_2019["Arrivals"].fillna(0)
mov_tur_com_2019["Presences"] = mov_tur_com_2019["Presences"].fillna(0)
mov_tur_com_2019.head(3)

,Year,Province,City,Month,From,Arrivals,Presences
0,2019,Città metropolitana Cagliari,ASSEMINI,1,Germania,6.0,10
1,2019,Città metropolitana Cagliari,ASSEMINI,1,Regno Unito,1.0,1
2,2019,Città metropolitana Cagliari,ASSEMINI,1,Portogallo,0.0,1


We merged nationalities in wide geographical areas except for national and regional provenance.

In [0]:
mov_tur_com_2019.From.replace(['Germania','Regno Unito','Portogallo','Spagna','Belgio','Norvegia','Svizzera (incluso Liechtenstein)','Francia','Paesi Bassi','Irlanda','Svezia','Danimarca','Lussemburgo','Austria','Malta','Finlandia','Islanda'],'Europe-CW', inplace=True)

In [0]:
mov_tur_com_2019.From.replace(['Polonia','Bulgaria','Slovenia','Grecia','Lituania','Repubblica Ceca','Romania','Ucraina','Moldavia','Altri Paesi Europei','Ungheria','Slovacchia','Croazia','Estonia','Lettonia', 'Bielorussia'],'Europe-E', inplace=True)

In [0]:
mov_tur_com_2019.From.replace(['Turchia','Giappone',"Altri Paesi dell'Asia",'Altri Paesi Asia Occidentale','Israele','India','Corea del Sud','Cipro'],'Asia', inplace=True)

In [0]:
mov_tur_com_2019.From.replace(['Piemonte','Lombardia','Veneto','Friuli-Venezia Giulia','Liguria','Emilia Romagna','Toscana','Umbria','Marche','Lazio','Abruzzo','Campania','Puglia','Basilicata','Calabria','Sicilia','Bolzano','Trento','Molise',"Valle d'Aosta",'Italy'],'Italy', inplace=True)

In [0]:
mov_tur_com_2019.From.replace(["Stati Uniti d'America",'Canada','Altri Paesi o territori del Nord Americani','Messico'],'North America', inplace=True)

In [0]:
mov_tur_com_2019.From.replace(['Australia','Nuova Zelanda',"Altri Paesi o territori dell'Oceania"],'Oceania', inplace=True)

In [0]:
mov_tur_com_2019.From.replace(['Altri Paesi Africa Mediterranea','Egitto',"Altri Paesi dell'Africa",'Sud Africa'],'Africa', inplace=True)

In [0]:
mov_tur_com_2019.From.replace(['Brasile','Venezuela','Altri Paesi Centro - Sud America','Argentina'],'South America', inplace=True)

In [0]:
mov_tur_com_2019.From.unique()

array(['Europe-CW', 'Asia', 'Europe-E', 'South America', 'Oceania',
       'Italy', 'Sardegna', 'Russia', 'North America', 'Cina', 'Africa',
       'Non specificato'], dtype=object)

In [0]:
mov_tur_com_2019.head(3)

,Year,Province,City,Month,From,Arrivals,Presences
0,2019,Città metropolitana Cagliari,ASSEMINI,1,Europe-CW,6.0,10
1,2019,Città metropolitana Cagliari,ASSEMINI,1,Europe-CW,1.0,1
2,2019,Città metropolitana Cagliari,ASSEMINI,1,Europe-CW,0.0,1


In [0]:
mov_tur_com_2019.dtypes

Year           int64
Province      object
City          object
Month         object
From          object
Arrivals     float64
Presences      int64
dtype: object

In [0]:
mov_tur_com_2019[["Arrivals"]] = mov_tur_com_2019[["Arrivals"]].astype("int")
mov_tur_com_2019.dtypes

Year          int64
Province     object
City         object
Month        object
From         object
Arrivals      int64
Presences     int64
dtype: object

In [0]:
mov_tur_com_2019_sum = mov_tur_com_2019.groupby(['Year','Province','City','From'],as_index=False).sum()
mov_tur_com_2019_sum.head(3)

,Year,Province,City,From,Arrivals,Presences
0,2019,Città metropolitana Cagliari,ASSEMINI,Africa,12,41
1,2019,Città metropolitana Cagliari,ASSEMINI,Asia,47,59
2,2019,Città metropolitana Cagliari,ASSEMINI,Cina,2,4


In [0]:
mov_tur_com_2019_sum = mov_tur_com_2019_sum[mov_tur_com_2019_sum.Province == 'Provincia di Sassari'].reset_index(drop=True)
mov_tur_com_2019_sum

,Year,Province,City,From,Arrivals,Presences
0,2019,Provincia di Sassari,AGGIUS,Europe-CW,560,1250
1,2019,Provincia di Sassari,AGGIUS,Europe-E,28,116
2,2019,Provincia di Sassari,AGGIUS,Italy,199,428
3,2019,Provincia di Sassari,AGGIUS,North America,16,23
4,2019,Provincia di Sassari,AGGIUS,Oceania,6,12
5,2019,Provincia di Sassari,AGGIUS,Russia,16,84
6,2019,Provincia di Sassari,AGGIUS,Sardegna,171,235
7,2019,Provincia di Sassari,AGGIUS,South America,6,12
8,2019,Provincia di Sassari,AGLIENTU,Africa,11,37
9,2019,Provincia di Sassari,AGLIENTU,Asia,27,150


In [0]:
mov_tur_com_2019_sum = mov_tur_com_2019_sum[mov_tur_com_2019_sum.From != 'Non specificato'].reset_index(drop=True)
mov_tur_com_2019_sum = mov_tur_com_2019_sum[mov_tur_com_2019_sum.City != 'Altri'].reset_index(drop=True)
mov_tur_com_2019_sum

,Year,Province,City,From,Arrivals,Presences
0,2019,Provincia di Sassari,AGGIUS,Europe-CW,560,1250
1,2019,Provincia di Sassari,AGGIUS,Europe-E,28,116
2,2019,Provincia di Sassari,AGGIUS,Italy,199,428
3,2019,Provincia di Sassari,AGGIUS,North America,16,23
4,2019,Provincia di Sassari,AGGIUS,Oceania,6,12
5,2019,Provincia di Sassari,AGGIUS,Russia,16,84
6,2019,Provincia di Sassari,AGGIUS,Sardegna,171,235
7,2019,Provincia di Sassari,AGGIUS,South America,6,12
8,2019,Provincia di Sassari,AGLIENTU,Africa,11,37
9,2019,Provincia di Sassari,AGLIENTU,Asia,27,150


We choose presences as key performance indicator.

In [0]:
pivot_mov_tur_com_2019_sum = pd.pivot_table(mov_tur_com_2019_sum, values=['Presences'], index=['City'],
                    columns=['From'], aggfunc=np.sum)
pivot_mov_tur_com_2019_sum = pivot_mov_tur_com_2019_sum.fillna(0)
pivot_mov_tur_com_2019_sum.head(3)

Presences                                                             \
From        Africa    Asia    Cina Europe-CW Europe-E     Italy North America   
City                                                                            
AGGIUS         0.0     0.0     0.0    1250.0    116.0     428.0          23.0   
AGLIENTU      37.0   150.0     3.0  104427.0  12842.0  169978.0         379.0   
ALGHERO     2088.0  6275.0  1293.0  671713.0  87220.0  295697.0       15627.0   

                                                 
From     Oceania  Russia Sardegna South America  
City                                             
AGGIUS      12.0    84.0    235.0          12.0  
AGLIENTU   173.0   269.0  14961.0         307.0  
ALGHERO   5445.0  8460.0  69584.0        8614.0

In [0]:
pivot_mov_tur_com_2019_sum.columns = pivot_mov_tur_com_2019_sum.columns.droplevel(0)
pivot_mov_tur_com_2019_sum = pivot_mov_tur_com_2019_sum.reset_index().rename_axis(None, axis=1)
pivot_mov_tur_com_2019_sum.head(3)

,City,Africa,Asia,Cina,Europe-CW,Europe-E,Italy,North America,Oceania,Russia,Sardegna,South America
0,AGGIUS,0.0,0.0,0.0,1250.0,116.0,428.0,23.0,12.0,84.0,235.0,12.0
1,AGLIENTU,37.0,150.0,3.0,104427.0,12842.0,169978.0,379.0,173.0,269.0,14961.0,307.0
2,ALGHERO,2088.0,6275.0,1293.0,671713.0,87220.0,295697.0,15627.0,5445.0,8460.0,69584.0,8614.0


In [0]:
pivot_mov_tur_com_2019_sum.sum()==0

City             False
Africa           False
Asia             False
Cina             False
Europe-CW        False
Europe-E         False
Italy            False
North America    False
Oceania          False
Russia           False
Sardegna         False
South America    False
dtype: bool

In [0]:
Test1 = pd.concat([pop['City'],pivot_mov_tur_com_2019_sum['City']], axis=1, sort=False)
Test1

,City,City
0,AGGIUS,AGGIUS
1,AGLIENTU,AGLIENTU
2,ALÀ DEI SARDI,ALGHERO
3,ALGHERO,ARZACHENA
4,ANELA,BADESI
5,ARDARA,BANARI
6,ARZACHENA,BENETUTTI
7,BADESI,BERCHIDDA
8,BANARI,BONNANARO
9,BENETUTTI,BONO


In [0]:
pivot_mov_tur_com_2019_sum.City.replace(['SASSARI'],'SASSARI, SS', inplace=True)

In [0]:
pivot_mov_tur_com_2019_sum.shape

(63, 12)

In [0]:
MOV_TOUR = pivot_mov_tur_com_2019_sum.iloc[0:,1:].apply(lambda x: x / x.sum())
MOV_TOUR.head(3)

,Africa,Asia,Cina,Europe-CW,Europe-E,Italy,North America,Oceania,Russia,Sardegna,South America
0,0.000000,0.000000,0.000000,0.000320,0.000282,0.000134,0.000248,0.000435,0.001025,0.000632,0.000208
1,0.002624,0.003415,0.000166,0.026702,0.031231,0.053347,0.004087,0.006278,0.003281,0.040260,0.005315
2,0.148096,0.142854,0.071559,0.171755,0.212112,0.092803,0.168536,0.197583,0.103201,0.187251,0.149124


In [0]:
MOV_TOUR = pd.concat([pivot_mov_tur_com_2019_sum['City'], MOV_TOUR], axis=1, sort=False)
MOV_TOUR

,City,Africa,Asia,Cina,Europe-CW,Europe-E,Italy,North America,Oceania,Russia,Sardegna,South America
0,AGGIUS,0.000000,0.000000,0.000000,3.196210e-04,0.000282,0.000134,0.000248,0.000435,0.001025,0.000632,0.000208
1,AGLIENTU,0.002624,0.003415,0.000166,2.670165e-02,0.031231,0.053347,0.004087,0.006278,0.003281,0.040260,0.005315
2,ALGHERO,0.148096,0.142854,0.071559,1.717549e-01,0.212112,0.092803,0.168536,0.197583,0.103201,0.187251,0.149124
3,ARZACHENA,0.248670,0.370464,0.074658,1.613171e-01,0.142496,0.122461,0.352096,0.322520,0.320691,0.091352,0.332820
4,BADESI,0.015604,0.009083,0.007582,2.656536e-02,0.026758,0.037168,0.013244,0.009616,0.026422,0.021907,0.005713
5,BANARI,0.000000,0.000023,0.000000,1.508611e-05,0.000068,0.000013,0.000000,0.000472,0.000000,0.000075,0.000000
6,BENETUTTI,0.000000,0.000000,0.000000,5.471911e-05,0.000090,0.000044,0.000173,0.000871,0.000000,0.000832,0.000000
7,BERCHIDDA,0.000071,0.000000,0.000332,7.798752e-05,0.000105,0.000384,0.000054,0.000000,0.000012,0.002608,0.000069
8,BONNANARO,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000010,0.000000,0.000000,0.000000,0.000065,0.000000
9,BONO,0.000000,0.000000,0.000000,3.068362e-06,0.000007,0.000026,0.000000,0.000000,0.000000,0.000371,0.000000


In [0]:
num_top_tourist = 10

for tourist in MOV_TOUR['City']:
    print("----"+tourist+"----")
    temp = MOV_TOUR[MOV_TOUR['City'] == tourist].T.reset_index()
    temp.columns = ['Nationality','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_tourist))
    print('\n')

----AGGIUS----
     Nationality  freq
0         Africa   0.0
1           Asia   0.0
2           Cina   0.0
3      Europe-CW   0.0
4       Europe-E   0.0
5          Italy   0.0
6  North America   0.0
7        Oceania   0.0
8         Russia   0.0
9       Sardegna   0.0


----AGLIENTU----
     Nationality  freq
0          Italy  0.05
1       Sardegna  0.04
2      Europe-CW  0.03
3       Europe-E  0.03
4        Oceania  0.01
5  South America  0.01
6         Africa  0.00
7           Asia  0.00
8           Cina  0.00
9  North America  0.00


----ALGHERO----
     Nationality  freq
0       Europe-E  0.21
1        Oceania  0.20
2       Sardegna  0.19
3      Europe-CW  0.17
4  North America  0.17
5         Africa  0.15
6  South America  0.15
7           Asia  0.14
8         Russia  0.10
9          Italy  0.09


----ARZACHENA----
     Nationality  freq
0           Asia  0.37
1  North America  0.35
2  South America  0.33
3        Oceania  0.32
4         Russia  0.32
5         Africa  0.25
6      E

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_nation(row, num_top_tourist):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_tourist]

Now let's create the new dataframe and display the top 10 accommodation for each city.

In [0]:
num_top_tourist = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top nationalities
columns = ['City']
for ind in np.arange(num_top_tourist):
    try:
        columns.append('{}{} Most Common Nationalities'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Nationalities'.format(ind+1))

# create a new dataframe
tourist_sorted = pd.DataFrame(columns=columns)
tourist_sorted['City'] = MOV_TOUR['City']

for ind in np.arange(MOV_TOUR.shape[0]):
    tourist_sorted.iloc[ind, 1:] = return_most_common_nation(MOV_TOUR.iloc[ind, :], num_top_tourist)

tourist_sorted

,City,1st Most Common Nationalities,2nd Most Common Nationalities,3rd Most Common Nationalities,4th Most Common Nationalities,5th Most Common Nationalities,6th Most Common Nationalities,7th Most Common Nationalities,8th Most Common Nationalities,9th Most Common Nationalities,10th Most Common Nationalities
0,AGGIUS,Russia,Sardegna,Oceania,Europe-CW,Europe-E,North America,South America,Italy,Cina,Asia
1,AGLIENTU,Italy,Sardegna,Europe-E,Europe-CW,Oceania,South America,North America,Asia,Russia,Africa
2,ALGHERO,Europe-E,Oceania,Sardegna,Europe-CW,North America,South America,Africa,Asia,Russia,Italy
3,ARZACHENA,Asia,North America,South America,Oceania,Russia,Africa,Europe-CW,Europe-E,Italy,Sardegna
4,BADESI,Italy,Europe-E,Europe-CW,Russia,Sardegna,Africa,North America,Oceania,Asia,Cina
5,BANARI,Oceania,Sardegna,Europe-E,Asia,Europe-CW,Italy,South America,Russia,North America,Cina
6,BENETUTTI,Oceania,Sardegna,North America,Europe-E,Europe-CW,Italy,South America,Russia,Cina,Asia
7,BERCHIDDA,Sardegna,Italy,Cina,Europe-E,Europe-CW,Africa,South America,North America,Russia,Oceania
8,BONNANARO,Sardegna,Italy,South America,Russia,Oceania,North America,Europe-E,Europe-CW,Cina,Asia
9,BONO,Sardegna,Italy,Europe-E,Europe-CW,South America,Russia,Oceania,North America,Cina,Asia


Run *k*-means to cluster the neighborhood into 5 clusters.

In [0]:
# set number of clusters
kclusters = 5

MOV_TOUR_clustering = MOV_TOUR.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(MOV_TOUR_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 1, 3, 4, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each city.

In [0]:
# add clustering labels
tourist_sorted.insert(0, 'Cluster Tour_Labels', kmeans.labels_)

tourist_merged = pop

# merge tourist_grouped with pop to add latitude/longitude for each city
tourist_merged = tourist_merged.join(tourist_sorted.set_index('City'), on='City')
tourist_merged.dropna(subset=['Cluster Tour_Labels'],inplace=True)
tourist_merged[["Cluster Tour_Labels"]] = tourist_merged[["Cluster Tour_Labels"]].astype("int")
tourist_merged.head(90) # check the last columns!

,City,Province,Population,Longitude,Latitude,Cluster Tour_Labels,1st Most Common Nationalities,2nd Most Common Nationalities,3rd Most Common Nationalities,4th Most Common Nationalities,5th Most Common Nationalities,6th Most Common Nationalities,7th Most Common Nationalities,8th Most Common Nationalities,9th Most Common Nationalities,10th Most Common Nationalities
0,AGGIUS,SASSARI,1436,9.065222,40.929345,0,Russia,Sardegna,Oceania,Europe-CW,Europe-E,North America,South America,Italy,Cina,Asia
1,AGLIENTU,SASSARI,1221,9.112719,41.079573,4,Italy,Sardegna,Europe-E,Europe-CW,Oceania,South America,North America,Asia,Russia,Africa
3,ALGHERO,SASSARI,43845,8.315147,40.560021,1,Europe-E,Oceania,Sardegna,Europe-CW,North America,South America,Africa,Asia,Russia,Italy
6,ARZACHENA,SASSARI,13816,9.387612,41.079898,3,Asia,North America,South America,Oceania,Russia,Africa,Europe-CW,Europe-E,Italy,Sardegna
7,BADESI,SASSARI,1 862,8.884216,40.965069,4,Italy,Europe-E,Europe-CW,Russia,Sardegna,Africa,North America,Oceania,Asia,Cina
8,BANARI,SASSARI,550,8.697371,40.572927,0,Oceania,Sardegna,Europe-E,Asia,Europe-CW,Italy,South America,Russia,North America,Cina
9,BENETUTTI,SASSARI,1793,9.168829,40.456461,0,Oceania,Sardegna,North America,Europe-E,Europe-CW,Italy,South America,Russia,Cina,Asia
10,BERCHIDDA,SASSARI,2690,9.164524,40.784067,0,Sardegna,Italy,Cina,Europe-E,Europe-CW,Africa,South America,North America,Russia,Oceania
12,BONNANARO,SASSARI,945,8.764420,40.533359,0,Sardegna,Italy,South America,Russia,Oceania,North America,Europe-E,Europe-CW,Cina,Asia
13,BONO,SASSARI,3500,9.029993,40.416830,0,Sardegna,Italy,Europe-E,Europe-CW,South America,Russia,Oceania,North America,Cina,Asia


Finally, let's visualize the resulting clusters.

In [0]:
# create map
map_tourist_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tourist_merged['Latitude'], tourist_merged['Longitude'], tourist_merged['City'], tourist_merged['Cluster Tour_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_tourist_clusters)
       
map_tourist_clusters

#### Cluster 1

For the great part we can find here inside cities with tourist coming from other part of the region, from Italy or from Europe.

In [0]:
tourist_merged.loc[tourist_merged['Cluster Tour_Labels'] == 0, tourist_merged.columns[[0] + list(range(5, tourist_merged.shape[1]))]]

,City,Cluster Tour_Labels,1st Most Common Nationalities,2nd Most Common Nationalities,3rd Most Common Nationalities,4th Most Common Nationalities,5th Most Common Nationalities,6th Most Common Nationalities,7th Most Common Nationalities,8th Most Common Nationalities,9th Most Common Nationalities,10th Most Common Nationalities
0,AGGIUS,0,Russia,Sardegna,Oceania,Europe-CW,Europe-E,North America,South America,Italy,Cina,Asia
8,BANARI,0,Oceania,Sardegna,Europe-E,Asia,Europe-CW,Italy,South America,Russia,North America,Cina
9,BENETUTTI,0,Oceania,Sardegna,North America,Europe-E,Europe-CW,Italy,South America,Russia,Cina,Asia
10,BERCHIDDA,0,Sardegna,Italy,Cina,Europe-E,Europe-CW,Africa,South America,North America,Russia,Oceania
12,BONNANARO,0,Sardegna,Italy,South America,Russia,Oceania,North America,Europe-E,Europe-CW,Cina,Asia
13,BONO,0,Sardegna,Italy,Europe-E,Europe-CW,South America,Russia,Oceania,North America,Cina,Asia
14,BONORVA,0,Sardegna,Italy,Europe-E,Europe-CW,South America,Russia,Oceania,North America,Cina,Asia
17,BOTTIDDA,0,North America,Sardegna,Italy,Europe-CW,South America,Russia,Oceania,Europe-E,Cina,Asia
18,BUDDUSÒ,0,Sardegna,Asia,Italy,Russia,Europe-E,Europe-CW,North America,South America,Oceania,Cina
20,BULTEI,0,Sardegna,Italy,South America,Russia,Oceania,North America,Europe-E,Europe-CW,Cina,Asia


#### Cluster 2

Alghero and Olbia have very various tourists.

In [0]:
tourist_merged.loc[tourist_merged['Cluster Tour_Labels'] == 1, tourist_merged.columns[[0] + list(range(5, tourist_merged.shape[1]))]]

,City,Cluster Tour_Labels,1st Most Common Nationalities,2nd Most Common Nationalities,3rd Most Common Nationalities,4th Most Common Nationalities,5th Most Common Nationalities,6th Most Common Nationalities,7th Most Common Nationalities,8th Most Common Nationalities,9th Most Common Nationalities,10th Most Common Nationalities
3,ALGHERO,1,Europe-E,Oceania,Sardegna,Europe-CW,North America,South America,Africa,Asia,Russia,Italy
52,OLBIA,1,South America,Asia,North America,Oceania,Africa,Russia,Sardegna,Europe-CW,Italy,Europe-E


#### Cluster 3

Tourism in Sassari is for business or immigration.

In [0]:
tourist_merged.loc[tourist_merged['Cluster Tour_Labels'] == 2, tourist_merged.columns[[0] + list(range(5, tourist_merged.shape[1]))]]

,City,Cluster Tour_Labels,1st Most Common Nationalities,2nd Most Common Nationalities,3rd Most Common Nationalities,4th Most Common Nationalities,5th Most Common Nationalities,6th Most Common Nationalities,7th Most Common Nationalities,8th Most Common Nationalities,9th Most Common Nationalities,10th Most Common Nationalities
72,"SASSARI, SS",2,Cina,Sardegna,Africa,South America,Asia,Italy,Europe-E,North America,Europe-CW,Russia


#### Cluster 4

Arzachena and its Costa Smeralda are a very international attraction.

In [0]:
tourist_merged.loc[tourist_merged['Cluster Tour_Labels'] == 3, tourist_merged.columns[[0] + list(range(5, tourist_merged.shape[1]))]]

,City,Cluster Tour_Labels,1st Most Common Nationalities,2nd Most Common Nationalities,3rd Most Common Nationalities,4th Most Common Nationalities,5th Most Common Nationalities,6th Most Common Nationalities,7th Most Common Nationalities,8th Most Common Nationalities,9th Most Common Nationalities,10th Most Common Nationalities
6,ARZACHENA,3,Asia,North America,South America,Oceania,Russia,Africa,Europe-CW,Europe-E,Italy,Sardegna


#### Cluster 5

This cluster is the most interesting. There are only coastal, but not all, cities and their tourist nationalities are mainly italian and european.

In [0]:
tourist_merged.loc[tourist_merged['Cluster Tour_Labels'] == 4, tourist_merged.columns[[0] + list(range(5, tourist_merged.shape[1]))]]

,City,Cluster Tour_Labels,1st Most Common Nationalities,2nd Most Common Nationalities,3rd Most Common Nationalities,4th Most Common Nationalities,5th Most Common Nationalities,6th Most Common Nationalities,7th Most Common Nationalities,8th Most Common Nationalities,9th Most Common Nationalities,10th Most Common Nationalities
1,AGLIENTU,4,Italy,Sardegna,Europe-E,Europe-CW,Oceania,South America,North America,Asia,Russia,Africa
7,BADESI,4,Italy,Europe-E,Europe-CW,Russia,Sardegna,Africa,North America,Oceania,Asia,Cina
19,BUDONI,4,Europe-CW,Italy,Africa,Sardegna,Russia,Europe-E,Oceania,Asia,North America,Cina
34,GOLFO ARANCI,4,Italy,Russia,Europe-CW,North America,Europe-E,Oceania,Asia,South America,Africa,Sardegna
38,LA MADDALENA,4,Europe-E,South America,Sardegna,Italy,Africa,Oceania,North America,Asia,Europe-CW,Russia
40,LOIRI PORTO SAN PAOLO,4,Africa,Europe-E,Europe-CW,Oceania,Italy,North America,Asia,Sardegna,Russia,South America
60,PALAU,4,Italy,Russia,Europe-CW,Europe-E,Africa,South America,North America,Oceania,Sardegna,Asia
68,SAN TEODORO,4,Italy,Europe-CW,Europe-E,Africa,Oceania,Sardegna,Asia,North America,Russia,South America
70,SANTA TERESA GALLURA,4,Europe-E,Russia,Italy,Europe-CW,Africa,Oceania,North America,Asia,South America,Sardegna
78,STINTINO,4,Italy,Asia,Sardegna,South America,Africa,Europe-E,North America,Oceania,Russia,Europe-CW


In the following table we summed up results of the clustering analysis.

In [0]:
cluster_pop = pop['City']
cluster_pop = pd.concat([cluster_pop, pop_merged["Cluster Labels"], accom_merged["Cluster Acc_Labels"], tourist_merged["Cluster Tour_Labels"]], axis=1, sort=False)
cluster_pop

,City,Cluster Labels,Cluster Acc_Labels,Cluster Tour_Labels
0,AGGIUS,1.0,0.0,0.0
1,AGLIENTU,1.0,0.0,4.0
2,ALÀ DEI SARDI,2.0,0.0,NaN
3,ALGHERO,1.0,2.0,1.0
4,ANELA,4.0,0.0,NaN
5,ARDARA,1.0,0.0,NaN
6,ARZACHENA,1.0,3.0,3.0
7,BADESI,1.0,0.0,4.0
8,BANARI,1.0,0.0,0.0
9,BENETUTTI,2.0,0.0,0.0
